<a href="https://colab.research.google.com/github/dbanerjee181/SAT-Solver/blob/main/dpll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# my code starts from here
# Reading a file in the DIMACS format and
# create the clause from it
# The Dimacs format goes as follows
# c means comment
# p cnf indicates the number of variable
# followed by number of clauses
# function signature is
# [clauses,clause_count,variable_count] = read_dimacs_file(filename)

import re


def read_dimacs_file(filename):
    """Reads a DIMACS file and returns the clauses and variable count."""
    clauses = []
    variable_count = 0
    clause_count = 0
    with open(filename, 'r') as file:
      for line in file:
        line = line.strip()
        if line.startswith(' ') or line.startswith('\t') or (not line):
          continue
        if line.startswith('0'):
          continue
        if line.startswith('c') or line.startswith('%'):
          continue
        if(re.match(r'p cnf',line)):
          variable_count = int(line.split()[2])
          clause_count = int(line.split()[3])
        else:
          line.strip()
          clause_list = [int(x) for x in line.split()[:-1]]
          clauses.append(clause_list)
    return clauses,variable_count,clause_count


In [2]:
from os import read
import re
def read_dimacs_file_dict(filename):
  with open(filename, 'r') as file:
    clauses = {}
    variable_count = 0
    clause_counter = 0
    for line in file:
      line = line.strip()
      if line.startswith(' ') or line.startswith('\t') or (not line):
        continue
      if line.startswith('0'):
        continue
      if line.startswith('c') or line.startswith('%'):
        continue
      if(re.match(r'p cnf',line)):
        variable_count = int(line.split()[2])
        clause_count = int(line.split()[3])

      else:
        line.strip()
        clause_list = [int(x) for x in line.split()[:-1]]
        clauses[clause_counter] = clause_list
        clause_counter+=1
  return clauses,variable_count,clause_count

clauses,var_c,clas_c = read_dimacs_file_dict("satTestFile.cnf")
print(clauses[0])

FileNotFoundError: [Errno 2] No such file or directory: 'satTestFile.cnf'

In [3]:
# Given the clauses and assignment this function checks if
# it there is any unit clause and then updates the assignment
# vector
# function signature
# check_unit_clause(clauses,assignment)
# This updates the assignment
# dictionary in place


def check_unit_clause(clauses, assignment):
  """Checks for unit clauses and updates the assignment vector."""

  for clause in clauses:
    if len(clause) == 1:
      variable = abs(clause[0])
      if clause[0] > 0:
        assignment[variable] = True
        clauses.remove(clause)
      else:
        assignment[variable] = False
        clauses.remove(clause)
  return clauses, assignment



In [24]:
def check_unit_clause_dict(clauses):
  """Checks for unit clauses."""
  unit_var = []
  clauses_copy = clauses.copy()
  for key,clause in clauses_copy.items():
    if len(clause) == 1:
      variable = abs(clause[0])
      value = True if clause[0] > 0 else False
      unit_var.append((key,variable,value))
      del clauses[key]
  return unit_var ,clauses

original_clauses = {0: [1],1: [2,4],2: [-1],3:[-3],4:[3]}
unit_clause,clauses = check_unit_clause_dict(original_clauses)
print(unit_clause)
print(clauses)

key,variable,value = unit_clause[0]
print(key)
print(variable)
print(value)

[(0, 1, True), (2, 1, False), (3, 3, False), (4, 3, True)]
{1: [2, 4]}
0
1
True


In [25]:
def conflict_analyze(original_clauses, unit_clause):
  """Checks for conflict and updates the assignment vector."""
  variables = [item[1] for item in unit_clause]
  keys = [item[0] for item in unit_clause]
  values = [item[2] for item in unit_clause ]
  duplicates_var = len(variables) != len(set(variables))
  conflict = []
  conflict_var = []
  count1 = 0
  if(duplicates_var):
    for var in variables:
      count1 +=1
      count2 = 0
      for var2 in variables:
        count2 +=1

        if(var == var2 and keys[count1-1]<keys[count2-1]):
          key1 = keys[count1-1]
          key2 = keys[count2-1]
          orig1 = original_clauses[key1]
          orig2 = original_clauses[key2]

          orig1 = [x for x in orig1 if abs(x) != var]
          orig2 = [x for x in orig2 if abs(x) != var]

          combined_list =list(dict.fromkeys(orig1 + orig2))
          conflict.append((key1,key2,combined_list))
          conflict_var.append(var)
  return conflict,conflict_var

original_clauses = {0: [1,-5,6],1: [2,5,6],2: [-1,6,7],3:[-3,7,-8],4:[3,9]}
conflict,conflict_var = conflict_analyze(original_clauses,unit_clause)
print(conflict)
print(conflict_var)

[(0, 2, [-5, 6, 7]), (3, 4, [7, -8, 9])]
[1, 3]


In [12]:
unit_clause = [(0, 1, True), (1, 2, True), (2, 1, False), (3, 3, False), (4, 3, True)]
def remove_conflict_var(unit_clause,conflict_var):
  return [(key,variable,value) for key, variable, value in unit_clause if variable not in conflict_var]

assignment = {10:True}
unit_clause = remove_conflict_var(unit_clause,conflict_var)
print(unit_clause)

[(1, 2, True)]


In [13]:
def check_Source_implication(original_clauses,unit_clause):
  unit_clause_copy = unit_clause.copy()
  source_list = []
  for index,(key,variable,value) in enumerate(unit_clause_copy):
    orig = original_clauses[key]
    orig1 = [x for x in orig if abs(x) != variable]
    source_list.append((variable,value,orig1))
  return source_list

original_clauses = {0: [1,-5,6],1: [2,5,6],2: [-1,6,7],3:[-3,7,-8],4:[3,9]}
source_list = check_Source_implication(original_clauses,unit_clause)
print(source_list)

[(2, True, [5, 6])]


In [21]:
def assignment(graph_instance,source_list):
  for index,(variable,value,source) in enumerate(source_list):
    graph_instance.add_node(variable,value)
    for element in source:
      graph_instance.add_source_edge(variable,element)
  return graph_instance

new_assig_graph = Graph()
new_assig_graph.add_node(5,True)
new_assig_graph.add_node(6,True)
print(source_list)
assignment(new_assig_graph,source_list)
print(new_assig_graph.graph)
elem= new_assig_graph.graph.get(5)
print(elem['source'])

[(2, True, [5, 6])]
{5: {'value': True, 'sink': [2], 'source': []}, 6: {'value': True, 'sink': [2], 'source': []}, 2: {'value': True, 'sink': [], 'source': [5, 6]}}
[]


In [22]:
def backtrack_delete(graph_instance,conflict_node):
  if conflict_node not in graph_instance.graph:
    return graph_instance
  if not graph_instance.graph[conflict_node]['sink']:
    source = graph_instance.graph[conflict_node]['source']
    graph_instance.delete_node(conflict_node)
    for source_node in source:
      backtrack_delete(graph_instance,source_node)
  else :
    pass
  return graph_instance

print(new_assig_graph.graph)
backtrack_delete(new_assig_graph,2)
print(new_assig_graph.graph)

{5: {'value': True, 'sink': [2], 'source': []}, 6: {'value': True, 'sink': [2], 'source': []}, 2: {'value': True, 'sink': [], 'source': [5, 6]}}
{}


In [13]:
def check_unit_clause_dict(clauses,original_clauses, graph_instance):
  """Checks for unit clauses and updates the assignment vector."""
  clauses_copy = clauses.copy()
  for key,clause in clauses_copy.items():
    if len(clause) == 1:
      variable = abs(clause[0])
      if variable not in graph_instance.graph:
        if clause[0] > 0 :
          graph_instance.add_node(variable,True)

        elif clause[0]<0 :
          graph_instance.add_node(variable,False)

        list1 = original_clauses[key]
        diff_elements = [x for x in list1 if x not in clause]
        for element in diff_elements:
          graph_instance.add_source_edge(variable,element)
        del clauses[key]
      elif variable in graph_instance.graph:
        print("conflict")
        orig_src = graph_instance[variable]["source"]
        implicant_src = original_clauses[key]
        diff_elements = [x for x in implicant_src if x not in clause]
        print(orig_src)
        print(diff_elements)

  return clauses ,graph_instance


new_assig_graph = Graph()
new_assig_graph.add_node(2,True)
new_assig_graph.add_node(3,True)
new_assig_graph.add_node(1,False)
original_clauses = {0: [1, 2, 3]}
clauses = {0: [1]}
assignment = {}
clauses , listo = check_unit_clause_dict(clauses, original_clauses,new_assig_graph)
print(listo.graph)


conflict


TypeError: 'Graph' object is not subscriptable

In [8]:
#Simplify the clause based on assignment
#Checking procedure example

def simplify(clauses, assignment):
  new_clauses = []
  for clause in clauses:
    new_clause = []
    satisfied = False
    for literal in clause:
      variable = abs(literal)
      if variable in assignment:
        if (literal > 0 and assignment[variable]) or \
           (literal < 0 and not assignment[variable]):
          satisfied = True
          break  # Clause is satisfied
        elif (literal > 0 and not assignment[variable]) or \
             (literal < 0 and assignment[variable]):
          continue #literal is false

      else:
        new_clause.append(literal)
        satisfied = False

    if not satisfied:
        new_clauses.append(new_clause)

  return new_clauses

clauses = [[1, 2], [-1, 3], [-2, -3]]
assignment = {1: True}
new_clauses = simplify(clauses, assignment)
print(new_clauses)

[[3], [-2, -3]]


In [20]:
#Simplify the clause based on assignment
#Checking procedure example

def simplify_graph(clauses, graph_instance):
  new_clauses = []
  for clause in clauses:
    new_clause = []
    satisfied = False
    for literal in clause:
      variable = abs(literal)
      if variable in graph_instance.graph:
        new_variable = graph_instance.graph[variable]["value"]
        if (literal > 0 and new_variable) or \
           (literal < 0 and not new_variable):
          satisfied = True
          break  # Clause is satisfied
        elif (literal > 0 and not new_variable) or \
             (literal < 0 and new_variable):
          continue #literal is false

      else:
        new_clause.append(literal)
        satisfied = False

    if not satisfied:
        new_clauses.append(new_clause)

  return new_clauses

clauses = [[1, 2], [-1, 3], [-2, -3]]
new_assig_graph = Graph()
new_assig_graph.add_node(1,True)
new_clauses = simplify_graph(clauses, new_assig_graph)
print(new_clauses)

[[3], [-2, -3]]


In [9]:
def find_pure_literal(clauses, assignment):
    literals = {}
    mono_literals = {}
    for clause in clauses:
        for literal in clause:
          variable = abs(literal)
          if variable not in assignment:
            if literal not in literals:
              literals[literal] = 0
            literals[literal]+=1
    for literal in literals:
      if -literal not in literals:
        mono_literals[literal] = literals[literal]

    return mono_literals

clauses = [[1, 2], [-1], [-3], [4], [4,5], [7,-1]]
assignment = {1: True, 2: True}
find_pure_literal(clauses, assignment)

{-3: 1, 4: 2, 5: 1, 7: 1}

In [10]:
def unassigned_literal(clauses):
    literals = {}
    for clause in clauses:
        for literal in clause:
          variable = abs(literal)
          if variable not in literals:
            literals[variable] = [0,0]
          if literal > 0 :
            literals[variable] = [literals[variable][0]+1,literals[variable][1]]
          elif literal < 0:
            literals[variable] = [literals[variable][0],literals[variable][1]+1]

    sorted_literals = dict(sorted(literals.items(), key=lambda item: sum(item[1]), reverse=True))
    return sorted_literals



clauses = [[1, 2], [3,-2], [1,-3],[-2],[-2],[-2]]
assignment = {1: True}
stats = unassigned_literal(clauses)
# Accessing the first element
first_literal = next(iter(stats))
print("First literal:", first_literal)

First literal: 2


In [11]:
import time

initial_assignment = {}
initial_clauses = []

def dpll(clauses, assignment={}, iterations=0,initial_assignment={},initial_clauses=[]):
  print(clauses)
  print(assignment)
  print(iterations)



  if(iterations == 0):
    if(assignment):
      initial_assignment = assignment.copy()
      initial_clauses = clauses.copy()

  print(f"Initial assignments : {initial_assignment}")
  print(f"Initial clauses : {initial_clauses}")

  pure_literals = find_pure_literal(clauses, assignment)
  #pure literal assignments

  for literal in pure_literals:
    variable = abs(literal)
    if literal > 0:
      assignment[variable] = True
    else:
      assignment[variable] = False

  new_clauses = simplify(clauses, assignment)
  # Check if all clauses are satisfied
  if not new_clauses:
    return assignment, iterations
  # Check if any clause is empty (unsatisfiable)

  if(any(not clause for clause in new_clauses)and bool(initial_assignment)):
    print("here")
    assignment = {}
    new_clauses = initial_clauses.copy()
    initial_assignment = {}
    initial_clauses = []

  if (any(not clause for clause in new_clauses)and bool(not(initial_assignment))):
    print("meeting this condition")
    return None, iterations

  stats = unassigned_literal(new_clauses)
  if not stats:
    return assignment, iterations
  priority_var = next(iter(stats))
  new_assignment = assignment.copy()
  if (stats[priority_var][0] >=  stats[priority_var][1]):
    new_assignment[priority_var] = True
  elif (stats[priority_var][0] < stats[priority_var][1]):
    new_assignment[priority_var] = False

  new_clauses_true = simplify(new_clauses, new_assignment)
  updated_clauses,unit_assignment = check_unit_clause(new_clauses_true, new_assignment)
  result_true = dpll(updated_clauses, unit_assignment, iterations + 1,initial_assignment,initial_clauses)
  if result_true:
    return result_true

  new_assignment[priority_var] = not new_assignment[priority_var]
  new_clauses_false = simplify(new_clauses, new_assignment)
  updated_clauses, unit_assignment = check_unit_clause(new_clauses_false, new_assignment)
  result_false = dpll(updated_clauses, unit_assignment, iterations + 1,initial_assignment,initial_clauses)
  if result_false:
    return result_false


  return None

start_time = time.time()  # Record the start time
#clauses,variable_count,clause_count  = read_dimacs_file("satTestFile.cnf")
#print(clauses)
clauses = [[-1, -2, -3], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-2, -3, -4], [-1, -2, -3],[-1,-2,3]]
assignment = {}
iterations = 0
result, iterations= dpll(clauses, assignment, iterations)

if not result:
  print("RESULT: UNSAT")
else:
  print("RESULT: SAT")
  print(sorted(result.items()))
  print("Time taken:", time.time() - start_time)
  print("Calls of DPLL:", iterations)

[[-1, -2, -3], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-2, -3, -4], [-1, -2, -3], [-1, -2, 3]]
{}
0
Initial assignments : {}
Initial clauses : []
RESULT: SAT
[(1, False), (2, False), (4, False)]
Time taken: 0.0004029273986816406
Calls of DPLL: 0


In [16]:

class Graph:
    def __init__(self):
        self.graph = {}

    def add_node(self, node, value):
        if node not in self.graph:
            self.graph[node] = {'value': value, 'sink': [], 'source':[]}  # Initialize node with value and empty neighbors list
        else:
            print(f"Node '{node}' already exists.")
            if(self.graph[node]['value'] != value):
              print(f"conflict detected.conflict clause: ")


    def add_source_edge(self, node1, node2):
        if node1 in self.graph and node2 in self.graph:
            self.graph[node1]['source'].append(node2)
            self.graph[node2]['sink'].append(node1)  # Assuming undirected graph

    def delete_node(self, node):
      if node in self.graph:
        # Remove node from neighbors' lists
        for neighbor in (self.graph[node]['source']):
          if node in (self.graph[neighbor]['sink']):
            self.graph[neighbor]['sink'].remove(node)
        for neighbor in (self.graph[node]['sink']):
          if node in (self.graph[neighbor]['source']):
            self.graph[neighbor]['source'].remove(node)
        # Remove node from the graph
        del self.graph[node]

    def traverse_sink(self, start_node):
        visited = set()
        queue = [start_node]
        visited.add(start_node)

        while queue:
          current_node = queue.pop(0)
          print(f"Node: {current_node}, Value: {self.graph[current_node]['value']}, Neighbors : {self.graph[current_node]['sink']}")
          for neighbor in self.graph[current_node]['sink']:
            if neighbor not in visited:
              visited.add(neighbor)
              queue.append(neighbor)

    def traverse_source(self, start_node):
        visited = set()
        queue = [start_node]
        visited.add(start_node)

        while queue:
          current_node = queue.pop(0)
          print(f"Node: {current_node}, Value: {self.graph[current_node]['value']}, Neighbors : {self.graph[current_node]['source']}")
          for neighbor in self.graph[current_node]['source']:
            if neighbor not in visited:
              visited.add(neighbor)
              queue.append(neighbor)


In [13]:
assig_graph = Graph()
def dpll_cdcl(clauses,assignment,iterations=0):
  pure_literals = find_pure_literal(clauses, assignment)
  for literal in pure_literals:
    variable = abs(literal)
    if literal > 0:
      assig_graph.add_node(variable,True)
    else:
      assig_graph.add_node(variable,False)
  new_clauses = simplify_graph(clauses, assig_graph)
  if not new_clauses:
    return assignment, iterations
  # Check if any clause is empty (unsatisfiable)

  if(any(not clause for clause in new_clauses)):
    return None, iterations

  stats = unassigned_literal(new_clauses)
  if not stats:
    return assignment, iterations
  priority_var = next(iter(stats))
  if (stats[priority_var][0] >=  stats[priority_var][1]):
    assig_graph.add_node(priority_var,True)
  elif (stats[priority_var][0] < stats[priority_var][1]):
    assig_graph.add_node(priority_var,False)

  new_clauses_true = simplify_graph(new_clauses, assig_graph)
  updated_clauses,unit_assignment = check_unit_clause(new_clauses_true, new_assignment)
  result_true = dpll_cdcl(updated_clauses, assig_graph)
  if result_true:
    return result_true
  new_assignment[priority_var] = not new_assignment[priority_var]
  assig_graph.add_node(priority_var,not new_assignment[priority_var])
  new_clauses_false = simplify(new_clauses, new_assignment)
  updated_clauses, unit_assignment = check_unit_clause(new_clauses_false, new_assignment)
  result_false = dpll_cdcl(updated_clauses, unit_assignment)
  if result_false:
    return result_false
  return None
clauses = [[-1, -2, -3], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-2, -3, -4], [-1, -2, -3],[-1,-2,3]]
assignment = {}
iterations = 0
result, iterations= dpll_cdcl(clauses, assignment, iterations)
print(assig_graph.graph)
if not result:
  print("RESULT: UNSAT")
else:
  print("RESULT: SAT")
  print(sorted(result.items()))

{1: {'value': False, 'sink': [], 'source': []}, 2: {'value': False, 'sink': [], 'source': []}, 4: {'value': False, 'sink': [], 'source': []}}
RESULT: SAT
[(1, False), (2, False), (4, False)]


In [ ]:
def main():
  [clauses,variable_count,clause_count] = read_dimacs_file("uf20-0999.cnf")
  print(f"The number of variables are {variable_count},The number of clauses are {clause_count}")
  print(clauses)
  assignment = stalmarck(clauses)
  print(assignment)
  clauses = [[1, 2],[2],[-1, 3]]
  assignment = {1: True, 3: True}
  check_unit_clause(clauses, assignment)
  print(assignment)


In [ ]:
if __name__ == "__main__":
  main()

The number of variables are 20,The number of clauses are 91
[[13, 15, -5], [5, -13, -9], [-2, -13, -9], [-16, 18, 19], [-6, 14, 5], [-7, 4, 11], [-15, 19, 14], [20, -3, -19], [-20, -9, -11], [2, -6, -10], [13, -6, 3], [9, 11, -8], [-9, -19, 7], [-17, -20, 12], [-17, 4, -16], [20, -5, -7], [-10, -4, 11], [5, 9, -1], [17, -1, 19], [-1, -2, -6], [15, 17, -19], [15, -14, 18], [-16, -15, 19], [-16, 6, -15], [-20, 5, -3], [-10, 20, 16], [-6, 17, -7], [7, 2, -16], [-18, 5, 13], [-17, 13, 12], [-14, -6, -12], [14, -2, -9], [3, -14, -17], [-1, 18, -6], [14, -18, -8], [7, -3, -19], [-18, -20, -5], [20, 12, 15], [5, 3, 15], [16, -6, -18], [8, 5, -18], [4, 6, -15], [6, 3, 4], [9, -11, -12], [12, 9, 5], [4, 18, -8], [16, -8, 1], [3, 1, -7], [15, -9, -4], [-5, -3, -10], [-16, -12, -19], [12, -3, -16], [4, -18, -6], [5, -7, -3], [15, -1, -5], [-16, 9, 10], [-9, 17, 5], [-2, 4, 10], [16, 9, -11], [1, -7, -15], [-20, -8, 3], [3, 9, 17], [-11, 9, 6], [8, 16, 19], [2, 8, -3], [-5, 15, 18], [1, 16, 2], [-